# Moving, splitting data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import random

In [ ]:
source_folder = "/content/drive/MyDrive/truck_annotations"
destination_folder = "labels"

os.makedirs(destination_folder, exist_ok=True)

files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

for file in files:
    shutil.copy(os.path.join(source_folder, file), os.path.join(destination_folder, file))

print(f"Moved {len(files)} files from {source_folder} to {destination_folder}.")


In [ ]:
source_folder = "/content/drive/MyDrive//truck_images"
destination_folder = "images"

os.makedirs(destination_folder, exist_ok=True)

files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

for file in files:
    shutil.copy(os.path.join(source_folder, file), os.path.join(destination_folder, file))

print(f"Moved {len(files)} files from {source_folder} to {destination_folder}.")

In [ ]:
images_folder = "images"
annotations_folder = "labels"
output_folder = "./"

train_images = os.path.join(output_folder, "train", "images")
train_annotations = os.path.join(output_folder, "train", "labels")
val_images = os.path.join(output_folder, "val", "images")
val_annotations = os.path.join(output_folder, "val", "labels")
test_images = os.path.join(output_folder, "test", "images")
test_annotations = os.path.join(output_folder, "test", "labels")

for folder in [train_images, train_annotations, val_images, val_annotations, test_images, test_annotations]:
    os.makedirs(folder, exist_ok=True)

image_files = sorted([f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png'))])
annotation_files = sorted([f for f in os.listdir(annotations_folder) if f.endswith('.txt')])

paired_files = [(img, ann) for img, ann in zip(image_files, annotation_files) if os.path.splitext(img)[0] == os.path.splitext(ann)[0]]

if not paired_files:
    raise ValueError("No matching image and annotation files found.")

random.shuffle(paired_files)

n = len(paired_files)
train_split = int(0.7 * n)
val_split = int(0.2 * n)

train_files = paired_files[:train_split]
val_files = paired_files[train_split:train_split + val_split]
test_files = paired_files[train_split + val_split:]

def copy_files(file_pairs, image_dest, annotation_dest):
    for img, ann in file_pairs:
        shutil.copy2(os.path.join(images_folder, img), image_dest)
        shutil.copy2(os.path.join(annotations_folder, ann), annotation_dest)

copy_files(train_files, train_images, train_annotations)
copy_files(val_files, val_images, val_annotations)
copy_files(test_files, test_images, test_annotations)

print("Data split completed!")

# Train


In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
import torch
device=["cuda" if torch.cuda.is_available() else "cpu"]

In [ ]:
model = YOLO('best.pt')

results = model.train(data='/content/data.yml', epochs=100, imgsz=640, batch=16)

In [ ]:
model.val()

In [ ]:
pred=model.predict("test/images", save=True, imgsz=640, conf=0.5)


# Saving results



In [ ]:
import zipfile

def zip_folder(folder_path, zip_filename):
  """Zips the specified folder.

  Args:
    folder_path: The path to the folder to zip.
    zip_filename: The name of the output zip file.
  """
  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
      for file in files:
        zipf.write(os.path.join(root, file),
                   os.path.relpath(os.path.join(root, file),
                                   os.path.join(folder_path, '..')))


folder_to_zip = "/content/runs/detect/train43"
zip_file_name = "/content/train43.zip"

zip_folder(folder_to_zip, zip_file_name)
print(f"Folder '{folder_to_zip}' zipped to '{zip_file_name}'")

In [ ]:
import shutil
shutil.copy("train4.zip", '/content/drive/MyDrive/runs_train4_backup')